<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-04-17 09:51:22--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  3.05MB/s    in 10m 33s 

2023-04-17 10:01:56 (2.19 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# method most_simmilar
wv_embeddings.most_similar("dog") 
# ОТВЕТ: Нет, но слово 'cats' на 4 месте

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    embed_list = []
    for word in tokenizer.tokenize(question.lower()):
        try:
            embed_list.append(embeddings[word])
        except (KeyError):
            continue
    return np.mean(embed_list, axis=0) if len(embed_list) > 0 else np.zeros(dim, dtype='float32')

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
question = "I love neural networks"
vector = question_to_vec(question, wv_embeddings, tokenizer, dim=200)

# третья компонента вектора
result = round(vector[2], 2)
result

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?
Ответ: возможный максимум Hits@47 = 1, а возможный минимум DCG@1 == 0, соотвественно макимум может быть в размере 1.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)
Ответ: DSC@10= 1/ log_2(1+9)*[9<=10]= 1/ log_2(10) = 0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = round(sum([np.zeros(1)[0] if rank > k else np.ones(1)[0] for rank in dup_ranks]) / len(dup_ranks),3)
    return hits_value

In [ ]:
def dcg_score(dup_ranks, k: int):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = round(sum([np.zeros(1)[0] if rank > k else math.log2(1 + rank)**(-1) for rank in dup_ranks]) / len(dup_ranks),3)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd
import math 

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]


# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0.0, 0.631, 0.631, 0.631]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
from google.colab import drive # Настроим доступ к данным
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!unzip -q /content/gdrive/MyDrive/DATA/stackoverflow_similar_questions.zip

Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


In [ ]:
validation_data[0]

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname',
 'Use a usercontrol from another project to current webpage',
 '~ Paths resolved differently after upgrading to ASP.NET 4',
 'Materialize datepicker - Rendering when an icon is clicked',
 'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
 'How can I analyze a confusion matrix?',
 'How do I declare a C array in Swift?',
 'Using rand() when flipping a coin and rolling a die',
 'Handling a JSON field with a special character in its name in Java',
 'React Native select row on ListView when push it',
 "Get 'creation_time' of video using ffmpeg and regex",
 'Does row exist and multiple where',
 "How to specify a classifier in a gradle dependency's dependency?",
 'Using $unwind on multiple do

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        embeddings: словарь векторных представлений слов
        tokenizer: токенизатор
        dim: размерность векторов
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    result = []
    q_vec = question_to_vec(question, embeddings, tokenizer)
    
    for idx, elem in enumerate(candidates):
        candidate_vec = question_to_vec(elem, embeddings, tokenizer)
        result.append((idx, elem, cosine_similarity(q_vec.reshape(1, -1), candidate_vec.reshape(1, -1))[0][0]))
        
    result.sort(key=lambda x: x[2], reverse=True)
    
    return [elem[:-1] for elem in deepcopy(result)]
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента? Ответ: 021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.276 | Hits@   1: 276.000
DCG@   5: 0.332 | Hits@   5: 386.000
DCG@  10: 0.349 | Hits@  10: 438.000
DCG@ 100: 0.395 | Hits@ 100: 671.000
DCG@ 500: 0.421 | Hits@ 500: 875.000
DCG@1000: 0.434 | Hits@1000: 1000.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preproc_nltk(text):
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])

In [ ]:
couple_list = [couple[0] + str(' ') + couple[1] for couple in train_data]
words = [preproc_nltk(question).split() for question in couple_list]

In [ ]:
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

In [ ]:
#Skip-Gram

In [ ]:
dcg_history = []
hits_history = []
tokenizer = MyTokenizer()
for window in range(1, 11):
    embeddings_trained = Word2Vec(words, 
                                 vector_size=200,        
                                 min_count=5,             
                                 window=window,    
                                 sg=1).wv # sg=1 - Skip-Gram
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    print(f'Window = {window}')
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
        dcg_history.append(dcg_score(wv_ranking, k))
        hits_history.append(hits_count(wv_ranking, k))
    print()

  0%|          | 0/3760 [00:00<?, ?it/s]

Window = 1
DCG@   1: 0.293 | Hits@   1: 293.000
DCG@   5: 0.362 | Hits@   5: 428.000
DCG@  10: 0.376 | Hits@  10: 470.000
DCG@ 100: 0.417 | Hits@ 100: 672.000
DCG@ 500: 0.438 | Hits@ 500: 838.000
DCG@1000: 0.455 | Hits@1000: 1000.000



  0%|          | 0/3760 [00:00<?, ?it/s]

Window = 2
DCG@   1: 0.308 | Hits@   1: 308.000
DCG@   5: 0.374 | Hits@   5: 433.000
DCG@  10: 0.393 | Hits@  10: 490.000
DCG@ 100: 0.430 | Hits@ 100: 676.000
DCG@ 500: 0.451 | Hits@ 500: 840.000
DCG@1000: 0.468 | Hits@1000: 1000.000



  0%|          | 0/3760 [00:00<?, ?it/s]

...

Window = 8
DCG@   1: 0.329 | Hits@   1: 329.000
DCG@   5: 0.399 | Hits@   5: 461.000
DCG@  10: 0.416 | Hits@  10: 512.000
DCG@ 100: 0.454 | Hits@ 100: 701.000
DCG@ 500: 0.474 | Hits@ 500: 858.000
DCG@1000: 0.489 | Hits@1000: 1000.000



  0%|          | 0/3760 [00:00<?, ?it/s]

Window = 9
DCG@   1: 0.328 | Hits@   1: 328.000
DCG@   5: 0.397 | Hits@   5: 457.000
DCG@  10: 0.415 | Hits@  10: 513.000
DCG@ 100: 0.454 | Hits@ 100: 706.000
DCG@ 500: 0.474 | Hits@ 500: 856.000
DCG@1000: 0.489 | Hits@1000: 1000.000



  0%|          | 0/3760 [00:00<?, ?it/s]

Window = 10
DCG@   1: 0.332 | Hits@   1: 332.000
DCG@   5: 0.398 | Hits@   5: 455.000
DCG@  10: 0.420 | Hits@  10: 523.000
DCG@ 100: 0.457 | Hits@ 100: 705.000
DCG@ 500: 0.476 | Hits@ 500: 854.000
DCG@1000: 0.491 | Hits@1000: 1000.000



In [ ]:
len(dcg_history)

60

In [ ]:
# Создаем пустой датафрейм с заданными колонками
df = pd.DataFrame(columns=['Window'] + ['DCG@{}'.format(k) for k in [1, 5, 10, 100, 500, 1000]] + ['Hits@{}'.format(k) for k in [1, 5, 10, 100, 500, 1000]] + ['SUM'])

# Заполняем датафрейм значениями из списков dcg_history и hits_history
for i, window in enumerate(range(1, 11)):
    row_values = [window] + dcg_history[i*6:(i+1)*6] + hits_history[i*6:(i+1)*6] + [sum(dcg_history[i*6:(i+1)*6]) + sum(hits_history[i*6:(i+1)*6])]
    df.loc[i] = row_values

In [ ]:
df

,Window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
0,1.0,0.293,0.362,0.376,0.417,0.438,0.455,0.293,0.428,0.470,0.672,0.838,1.0,3703.341
1,2.0,0.308,0.374,0.393,0.430,0.451,0.468,0.308,0.433,0.490,0.676,0.840,1.0,3749.424
2,3.0,0.314,0.380,0.398,0.436,0.458,0.474,0.314,0.437,0.495,0.678,0.843,1.0,3769.460
3,4.0,0.314,0.384,0.403,0.440,0.460,0.477,0.314,0.445,0.504,0.685,0.844,1.0,3794.478
4,5.0,0.328,0.394,0.411,0.449,0.471,0.486,0.328,0.451,0.503,0.687,0.851,1.0,3822.539
5,6.0,0.324,0.394,0.413,0.450,0.470,0.486,0.324,0.454,0.513,0.697,0.852,1.0,3842.537
6,7.0,0.327,0.397,0.417,0.453,0.473,0.488,0.327,0.456,0.519,0.703,0.854,1.0,3861.555
7,8.0,0.329,0.399,0.416,0.454,0.474,0.489,0.329,0.461,0.512,0.701,0.858,1.0,3863.561
8,9.0,0.328,0.397,0.415,0.454,0.474,0.489,0.328,0.457,0.513,0.706,0.856,1.0,3862.557
9,10.0,0.332,0.398,0.420,0.457,0.476,0.491,0.332,0.455,0.523,0.705,0.854,1.0,3871.574


In [ ]:
df[df['Hits@1']==df['Hits@1'].max()] # лучшее значение окна 

,Window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
9,10.0,0.332,0.398,0.42,0.457,0.476,0.491,0.332,0.455,0.523,0.705,0.854,1.0,3871.574


In [ ]:
df[df['SUM']==df['SUM'].max()] # лучшее значение окна при сумме всех метрик

,Window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
9,10.0,0.332,0.398,0.42,0.457,0.476,0.491,0.332,0.455,0.523,0.705,0.854,1.0,3871.574


In [ ]:
#CBOW

In [ ]:
dcg_history = []
hits_history = []
tokenizer = MyTokenizer()
for window in range(1, 11):
    embeddings_trained = Word2Vec(words, 
                                 vector_size=200,        
                                 min_count=5,             
                                 window=window,    
                                 sg=0).wv # sg=0 - CBOW
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    print(f'Window = {window}')
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
        dcg_history.append(dcg_score(wv_ranking, k))
        hits_history.append(hits_count(wv_ranking, k))
    print()

  0%|          | 0/3760 [00:00<?, ?it/s]

Window = 1
DCG@   1: 0.273 | Hits@   1: 0.273
DCG@   5: 0.334 | Hits@   5: 0.390
DCG@  10: 0.353 | Hits@  10: 0.448
DCG@ 100: 0.396 | Hits@ 100: 0.659
DCG@ 500: 0.420 | Hits@ 500: 0.847
DCG@1000: 0.436 | Hits@1000: 1.000



  0%|          | 0/3760 [00:00<?, ?it/s]

Window = 2
DCG@   1: 0.261 | Hits@   1: 0.261
DCG@   5: 0.324 | Hits@   5: 0.380
DCG@  10: 0.349 | Hits@  10: 0.455
DCG@ 100: 0.393 | Hits@ 100: 0.673
DCG@ 500: 0.418 | Hits@ 500: 0.869
DCG@1000: 0.432 | Hits@1000: 1.000



  0%|          | 0/3760 [00:00<?, ?it/s]

...

Window = 8
DCG@   1: 0.269 | Hits@   1: 0.269
DCG@   5: 0.334 | Hits@   5: 0.392
DCG@  10: 0.358 | Hits@  10: 0.466
DCG@ 100: 0.403 | Hits@ 100: 0.688
DCG@ 500: 0.427 | Hits@ 500: 0.871
DCG@1000: 0.441 | Hits@1000: 1.000



  0%|          | 0/3760 [00:00<?, ?it/s]

Window = 9
DCG@   1: 0.267 | Hits@   1: 0.267
DCG@   5: 0.336 | Hits@   5: 0.397
DCG@  10: 0.357 | Hits@  10: 0.463
DCG@ 100: 0.404 | Hits@ 100: 0.691
DCG@ 500: 0.427 | Hits@ 500: 0.873
DCG@1000: 0.441 | Hits@1000: 1.000



  0%|          | 0/3760 [00:00<?, ?it/s]

Window = 10
DCG@   1: 0.275 | Hits@   1: 0.275
DCG@   5: 0.340 | Hits@   5: 0.399
DCG@  10: 0.359 | Hits@  10: 0.459
DCG@ 100: 0.408 | Hits@ 100: 0.694
DCG@ 500: 0.430 | Hits@ 500: 0.870
DCG@1000: 0.444 | Hits@1000: 1.000



In [ ]:
# Создаем пустой датафрейм с заданными колонками
df2 = pd.DataFrame(columns=['Window'] + ['DCG@{}'.format(k) for k in [1, 5, 10, 100, 500, 1000]] + ['Hits@{}'.format(k) for k in [1, 5, 10, 100, 500, 1000]] + ['SUM'])

# Заполняем датафрейм значениями из списков dcg_history и hits_history
for i, window in enumerate(range(1, 11)):
    row_values = [window] + dcg_history[i*6:(i+1)*6] + hits_history[i*6:(i+1)*6] + [sum(dcg_history[i*6:(i+1)*6]) + sum(hits_history[i*6:(i+1)*6])]
    df2.loc[i] = row_values

In [ ]:
df2

,Window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
0,1.0,0.273,0.334,0.353,0.396,0.420,0.436,0.273,0.390,0.448,0.659,0.847,1.0,5.829
1,2.0,0.261,0.324,0.349,0.393,0.418,0.432,0.261,0.380,0.455,0.673,0.869,1.0,5.815
2,3.0,0.269,0.333,0.353,0.399,0.424,0.437,0.269,0.395,0.455,0.682,0.876,1.0,5.892
3,4.0,0.265,0.333,0.352,0.399,0.424,0.437,0.265,0.395,0.456,0.686,0.877,1.0,5.889
4,5.0,0.263,0.330,0.352,0.398,0.422,0.436,0.263,0.390,0.458,0.687,0.874,1.0,5.873
5,6.0,0.271,0.334,0.356,0.402,0.426,0.439,0.271,0.392,0.461,0.684,0.871,1.0,5.907
6,7.0,0.275,0.340,0.359,0.406,0.430,0.443,0.275,0.401,0.462,0.693,0.877,1.0,5.961
7,8.0,0.269,0.334,0.358,0.403,0.427,0.441,0.269,0.392,0.466,0.688,0.871,1.0,5.918
8,9.0,0.267,0.336,0.357,0.404,0.427,0.441,0.267,0.397,0.463,0.691,0.873,1.0,5.923
9,10.0,0.275,0.340,0.359,0.408,0.430,0.444,0.275,0.399,0.459,0.694,0.870,1.0,5.953


In [ ]:
df2[df2['Hits@1']==df2['Hits@1'].max()] # лучшее значение окна 

,Window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
6,7.0,0.275,0.34,0.359,0.406,0.43,0.443,0.000275,0.000401,0.000462,0.000693,0.000877,0.001,5.961
9,10.0,0.275,0.34,0.359,0.408,0.43,0.444,0.000275,0.000399,0.000459,0.000694,0.000870,0.001,5.953


In [ ]:
df2[df2['SUM']==df2['SUM'].max()] # лучшее значение окна при сумме метрик

,Window,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
6,7.0,0.275,0.34,0.359,0.406,0.43,0.443,0.000275,0.000401,0.000462,0.000693,0.000877,0.001,5.961


In [ ]:
# выберим лучший метод обучения и размер окна.
# CBOW, window=7

In [ ]:
embeddings_trained = Word2Vec(words, 
            vector_size=200,         
            min_count=5,             
            window=7,               
            workers=-1,               
            sg=0).wv 

In [ ]:
from nltk.tokenize import WordPunctTokenizer, TreebankWordTokenizer

In [ ]:
import logging
logging.getLogger("gensim.models.word2vec").setLevel(logging.ERROR)
tokenizers = [TreebankWordTokenizer(), WordPunctTokenizer(), MyTokenizer()]
dcg_history = []
hits_history = []

for tokenizer in tokenizers:
    embeddings_trained = Word2Vec(words, 
                                  vector_size=200, 
                                  min_count=5, 
                                  window=7, 
                                  workers=-1, 
                                  sg=0).wv

    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(validation_data):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)

    print(f'Tokenizer: {type(tokenizer).__name__}')
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
        dcg_history.append(dcg_score(wv_ranking, k))
        hits_history.append(hits_count(wv_ranking, k))
    print()

Tokenizer: TreebankWordTokenizer


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.304 | Hits@   1: 0.304
DCG@   5: 0.366 | Hits@   5: 0.421
DCG@  10: 0.387 | Hits@  10: 0.484
DCG@ 100: 0.426 | Hits@ 100: 0.681
DCG@ 500: 0.447 | Hits@ 500: 0.846
DCG@1000: 0.463 | Hits@1000: 1.000

Tokenizer: WordPunctTokenizer


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.313 | Hits@   1: 0.313
DCG@   5: 0.378 | Hits@   5: 0.432
DCG@  10: 0.397 | Hits@  10: 0.491
DCG@ 100: 0.438 | Hits@ 100: 0.694
DCG@ 500: 0.460 | Hits@ 500: 0.864
DCG@1000: 0.474 | Hits@1000: 1.000

Tokenizer: MyTokenizer


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.359 | Hits@   1: 0.359
DCG@   5: 0.441 | Hits@   5: 0.516
DCG@  10: 0.458 | Hits@  10: 0.568
DCG@ 100: 0.491 | Hits@ 100: 0.726
DCG@ 500: 0.510 | Hits@ 500: 0.874
DCG@1000: 0.523 | Hits@1000: 1.000



In [ ]:
# теперь используем номализацию

In [ ]:
def preproc_nltk_lemma(text):
    text = ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])
    return MyTokenizer().tokenize(text)
words_lemma = [preproc_nltk_lemma(question) for question in couple_list]

In [ ]:
for i in range(5): 
    print(words_lemma[i])
    print()

['converting', 'string', 'list', 'convert', 'google', 'results', 'object', 'pure', 'js', 'python', 'object']

['html', '5', 'canvas', 'javascript', 'use', 'making', 'interactive', 'drawing', 'tool', 'event', 'handling', 'geometries', 'three', 'js']

['sending', 'array', 'via', 'ajax', 'fails', 'getting', 'list', 'items', 'unordered', 'list', 'php']

['insert', 'cookiecollection', 'cookiecontainer', 'c', 'create', 'cookie', 'string', 'send']

['updating', 'one', 'element', 'bound', 'observable', 'collection', 'wpf', 'update', 'changes', 'list', 'item', 'list']



In [ ]:
embeddings_trained_lemma = Word2Vec(words_lemma, 
            vector_size=200,         
            min_count=5,             
            window=7,               
            workers=-1,              
            sg=0).wv   

In [ ]:
tokenizers = [TreebankWordTokenizer(), WordPunctTokenizer(), MyTokenizer()]
count=0

for tokenizer in tokenizers:
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(validation_data):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained_lemma, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    if count < 2:
        print(str(tokenizer).split(".")[1].split(" ")[0])
    else:
        print(str(tokenizer).split("(")[0])
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print()
    count+=1

tokenize
DCG@   1: 0.228 | Hits@   1: 0.228
DCG@   5: 0.288 | Hits@   5: 0.341
DCG@  10: 0.305 | Hits@  10: 0.396
DCG@ 100: 0.346 | Hits@ 100: 0.603
DCG@ 500: 0.376 | Hits@ 500: 0.838
DCG@1000: 0.393 | Hits@1000: 1.000

UNICODE|re
DCG@   1: 0.250 | Hits@   1: 0.250
DCG@   5: 0.311 | Hits@   5: 0.368
DCG@  10: 0.326 | Hits@  10: 0.416
DCG@ 100: 0.367 | Hits@ 100: 0.620
DCG@ 500: 0.398 | Hits@ 500: 0.856
DCG@1000: 0.413 | Hits@1000: 1.000

DCG@   1: 0.250 | Hits@   1: 0.250
DCG@   5: 0.311 | Hits@   5: 0.368
DCG@  10: 0.326 | Hits@  10: 0.416
DCG@ 100: 0.367 | Hits@ 100: 0.620
DCG@ 500: 0.398 | Hits@ 500: 0.856
DCG@1000: 0.413 | Hits@1000: 1.000



Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи? 
* Предложите свой подход к решению задачи.


## Вывод:
* Как ни странно MyTokenizer, дает лучшее качество, возможно вопросы без лишних символов и чем проще токенайзер тем лучше в наших данных.
* Нормализация не помогла
* CBOW лучше справляется, возможно контекст в вопросах в этой задаче важнее.
* Модель обучилась недостаточно и следует проверить другие гиперпараметры.
* Можно покрутить гипермпараметры, дообучить модель, попробовать другие токенайзеры.
